# Training FCN Model
This script performs the training of a network after the dataset has been constructed. Four modules are used in this training:

1. `input_data.py`: This module loads in the training and testing data that was created, and prepares it so that it can be fed into the neural network for training.
2. `models.py`: this module contains different types of neural network architectures. One can choose a model from this script and train it.
3. `training_utilities.py`: this module contains functions to construct models, set up diagnostic files, and perform the training on the model.
4. `accuracy.py`: The accuracy that the FCN outputs looks at the predicted label and evaluated label, and does a pixelwise comparison (#pixels set to the correct value)/(#pixels in the image). This isn't the best way to determine the success of our model. Rather, we want to look at a metric that shows that the model is detecting defects (where each defect consists of multiple pixels). Hence we use measures such as the recall, which is the number of true positive defect detections TN over the true positives plus the false negatives (FN), R = TN/(TN+FN). This module helps in calculating such metrics

First, we import these modules

In [1]:
from input_data import *
from models import *
from training_utilities import *

## Parameters ##
Next, we need to provide parameter information based on the data and model we're using.

**parent_dir**: This is the path to the "stem-learning" code

**data_dir**: This is the path to the data that we created in the preprocessing section

**sess_name**: we will create a folder in the "results" directory called session_name, where all the output will be stored

**N**: This is the pixel width/height of the input images (note we're assuming a square image)

**k_fac**: this is a factor that describes how many channels we want per layer in our FCNs. Whatever the default value is per layer, it is multiplied by k_fac.

**nb_classes**: this is the number of labels that we are learning at once. For example, if our data is just the "2Te" labels, then nb_classes = 2 (2Te and no defect). 

**dropout**: This is the dropout rate at the end of our FCN

**num_steps**: The total number of steps to train on (-1 for infinite loop)

In [2]:
parent_dir = 'C:/Users/abidk/Dropbox/Development/programs/stem-learning/'
data_dir   = parent_dir + 'data/WSe/sim_50_gan/parsed_label_1vacancy/'
sess_name  = '1vacancy'
N          = 256
k_fac      = 16
nb_classes = 2
num_steps  = 500

The variables below are then created to locate the directories that we'll be storing our data

In [3]:
from os import makedirs
sess_dir = parent_dir + "results/" + sess_name + "/"
makedirs(sess_dir, exist_ok=True)

model_weights_fn = sess_dir + "weights.h5"
model_fn         = sess_dir + "model.json"
diagnostics_fn   = sess_dir + "diagnostics.dat"

Now we create the model and set up a diagnostics file

In [4]:
model = construct_model(N, k_fac, nb_classes, sess_dir, model_fn, model_weights_fn)
step = setup_diagnostics(diagnostics_fn)

loading weights
continuing session with step79


C:\Users\abidk\AppData\Roaming\Python\Python39\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Finally, we train

In [5]:
train(step, data_dir, N, nb_classes, model, diagnostics_fn, model_weights_fn, num_steps=num_steps)

training step: 79	training file: train_00124.p
	grabbing data
	done
32/32 - 13s - loss: -7.1894e+01 - accuracy: 0.9954 - val_loss: -5.8897e+01 - val_accuracy: 0.9960
	calculating accuracy
0.0


C:\Users\abidk\Dropbox\Development\programs\stem-learning\code\2_trainer\accuracy.py:7: RuntimeWarning: invalid value encountered in true_divide
  conv_label_img = (label_img - np.min(label_img))/np.ptp(label_img)


	done
TP = 0, FP = 0, FN = 16253, TN = 497507
recall = 0.0, precision = -1, F1 = -1, bal_acc = 0.5
training step: 80	training file: train_00033.p
	grabbing data
	done
32/32 - 7s - loss: -7.2813e+01 - accuracy: 0.9942 - val_loss: -7.8962e+01 - val_accuracy: 0.9960
	calculating accuracy
0.0
	done
TP = 0, FP = 0, FN = 16253, TN = 497507
recall = 0.0, precision = -1, F1 = -1, bal_acc = 0.5
training step: 81	training file: train_00086.p
	grabbing data
	done
32/32 - 7s - loss: -7.4123e+01 - accuracy: 0.9965 - val_loss: -8.6290e+01 - val_accuracy: 0.9960
	calculating accuracy
0.0
	done
TP = 0, FP = 0, FN = 16253, TN = 497507
recall = 0.0, precision = -1, F1 = -1, bal_acc = 0.5
training step: 82	training file: train_00084.p
	grabbing data
	done
32/32 - 7s - loss: -7.5224e+01 - accuracy: 0.9953 - val_loss: -8.8938e+01 - val_accuracy: 0.9960
	calculating accuracy
0.0
	done
TP = 0, FP = 0, FN = 16253, TN = 497507
recall = 0.0, precision = -1, F1 = -1, bal_acc = 0.5
training step: 83	training file

ResourceExhaustedError:  OOM when allocating tensor with shape[32,16,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/batch_normalization_29/FusedBatchNormV3 (defined at \Dropbox\Development\programs\stem-learning\code\2_trainer\training_utilities.py:121) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_7866]

Function call stack:
train_function
